In [41]:
# install Requirements.txt

In [42]:
import getpass
import os
from dotenv import load_dotenv

In [43]:
# load env and keys for langsmith and groq
# Prior to this you are expected to visit langsmith and groq websites and get your API keys ready
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"



if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")


In [44]:
from langchain.chat_models import init_chat_model

In [45]:
# make sure you are accessing a valid model, feel free to change the model name
model = init_chat_model("Llama-3.3-70B-Versatile", model_provider="groq")

In [46]:
from langchain_core.messages import HumanMessage

In [47]:
model.invoke([HumanMessage(content="Hi! I'm sagar")])

AIMessage(content='Hello Sagar! How are you today? Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 41, 'total_tokens': 66, 'completion_time': 0.061812261, 'prompt_time': 0.001883603, 'queue_time': 0.048243627, 'total_time': 0.063695864}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c81a48c-53ec-403b-8ea6-bf639f494dc0-0', usage_metadata={'input_tokens': 41, 'output_tokens': 25, 'total_tokens': 66})

In [48]:
model.invoke([HumanMessage('what is my name?')])

AIMessage(content="I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm here to help answer your questions and provide information, but I don't have any personal knowledge about you. If you'd like to share your name, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 40, 'total_tokens': 109, 'completion_time': 0.138243841, 'prompt_time': 0.001804163, 'queue_time': 0.048619017, 'total_time': 0.140048004}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--facc7b04-76a4-4449-8e6e-d11860a9e519-0', usage_metadata={'input_tokens': 40, 'output_tokens': 69, 'total_tokens': 109})

In [49]:
# models response is not based on the previous conversation, as it is not taking previous conversation as input we will now pass entire conversation as input to model

In [50]:
from langchain_core.messages import AIMessage

In [51]:
model.invoke(
    [
        HumanMessage("Hi, My name is Sagar"),
        AIMessage("Hello Sagar! How can I assist today?"),
        HumanMessage("What is my name?")
    ]
)

AIMessage(content='Your name is Sagar.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 67, 'total_tokens': 74, 'completion_time': 0.019853532, 'prompt_time': 0.003250479, 'queue_time': 0.048984071, 'total_time': 0.023104011}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--20b824cd-331c-466a-8986-2c72f5f10c40-0', usage_metadata={'input_tokens': 67, 'output_tokens': 7, 'total_tokens': 74})

Message persistence

In [52]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [53]:
# define new workflow
workflow = StateGraph(state_schema=MessagesState)

In [54]:
# define function to call model
def model_call(state: MessagesState):
    resposne = model.invoke(state['messages'])
    return {'messages': resposne}

In [55]:
# define the nodes in graph
workflow.add_edge(START, "model")
workflow.add_node("model", model_call)

In [56]:
# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [57]:
# create configurable for passing it to models, It supports multiple conversations threads with single application used by multiple users

config = {"configurable":{"thread_id":"abc123"}}

In [58]:
input_message = [HumanMessage('Hi, My name is Sagar')]
output = app.invoke({"messages": input_message}, config)
print(output)

{'messages': [HumanMessage(content='Hi, My name is Sagar', additional_kwargs={}, response_metadata={}, id='3778bbb4-adc9-45ae-829b-e59a7f4e8531'), AIMessage(content="Hello Sagar, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 42, 'total_tokens': 69, 'completion_time': 0.042871607, 'prompt_time': 0.002023039, 'queue_time': 0.04836656, 'total_time': 0.044894646}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7602f84b-7acc-4237-b213-95fcb8f5057f-0', usage_metadata={'input_tokens': 42, 'output_tokens': 27, 'total_tokens': 69})]}


In [59]:
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hello Sagar, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [60]:
input_message = [HumanMessage('what is my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

Your name is Sagar. You told me that when we started chatting.
None


In [61]:
# out models now started remembering things, now if we change config it will start a fresh chat

In [62]:
config = {"configurable": {"thread_id":"abc12345"}}
input_message = [HumanMessage('what is my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

I don't have any information about your name. I'm a large language model, I don't have the ability to know your personal details or identity. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to chat with you and use it in our conversation!
None


In [63]:
# let me check if model remembers out conversation with previous thread id
config = {"configurable": {"thread_id":"abc123"}}
input_message = [HumanMessage('Do you remember my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

Yes, I remember your name. It's Sagar.
None


In [68]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


input_message = [HumanMessage('can you describe our conversation?')]
# Async invocation:
output = await app.ainvoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

We've had a simple conversation so far. You introduced yourself, telling me that your name is Sagar. Then, you asked me to recall your name a few times, and I confirmed that I remembered it correctly. You also asked me to describe our conversation, which I'm doing now. That's the entirety of our conversation so far.
